In [34]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'time-series-lstm-pytorch'

role = sagemaker.get_execution_role()

In [35]:
role

'arn:aws:iam::401598941748:role/service-role/AmazonSageMaker-ExecutionRole-20200818T220388'

In [36]:
input_data = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/data', bucket=bucket, key_prefix=prefix)

In [37]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    instance_count=1,
                    instance_type='ml.m4.xlarge',
                    py_version="py3",
                    hyperparameters={
                        'epochs': 100,
                        'input_size': 1,
                        'hidden_size': 2
                    })

In [38]:
import pandas as pd

In [39]:
estimator.fit({'training': input_data})

2021-10-26 09:33:22 Starting - Starting the training job...
2021-10-26 09:33:48 Starting - Launching requested ML instancesProfilerReport-1635240802: InProgress
......
2021-10-26 09:34:48 Starting - Preparing the instances for training...............
2021-10-26 09:37:08 Downloading - Downloading input data
2021-10-26 09:37:08 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-26 09:37:28,913 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-10-26 09:37:28,916 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-10-26 09:37:28,930 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-26 09:37:28,942 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-26 09:37:29,473 sagemaker-containers INFO     Module train does not provide a setup.py

      Successfully uninstalled torch-0.4.0

2021-10-26 09:39:10 Uploading - Uploading generated training modelSuccessfully installed future-0.18.2 torch-1.5.1
1.5.1
Using device cpu.
Get train data loader.
Model loaded with num_classes 1, input_size 1, hidden_size 2, num_layers 1.
Epoch: 10, BCELoss: 80.87599435055031
Epoch: 20, BCELoss: 66.59032452781246
Epoch: 30, BCELoss: 58.37337752108304
Epoch: 40, BCELoss: 54.94467584142145
Epoch: 50, BCELoss: 50.56457840721562
Epoch: 60, BCELoss: 49.01130623187659
Epoch: 70, BCELoss: 49.753952431228925
Epoch: 80, BCELoss: 47.798461149323664
Epoch: 90, BCELoss: 47.5180895913322
Epoch: 100, BCELoss: 47.570036708184006
2021-10-26 09:39:08,655 sagemaker-containers INFO     Reporting training SUCCESS

2021-10-26 09:39:29 Completed - Training job completed
ProfilerReport-1635240802: NoIssuesFound
Training seconds: 138
Billable seconds: 138


In [26]:
predictor=estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-----------------!

In [27]:
from train.predict import get_train_data_loader

In [28]:
test_loader = get_train_data_loader(4, '/home/ec2-user/SageMaker/data')

Get train data loader.


In [29]:
import torch

In [30]:
for test_data_X, test_data_Y in test_loader:
    input_data = test_data_X
    input_data = input_data.unsqueeze(-1)
    #print(input_data, type(input_data), input_data.size())
    output = predictor.predict(input_data)
#     output = torch.from_numpy(output).float()
    print('Predicted OUTPUT = ', output, type(output))
    print('Actual OUTPUT = ', test_data_Y, type(test_data_Y))

Predicted OUTPUT =  [[7.083995 ]
 [7.083894 ]
 [7.083986 ]
 [7.0815587]] <class 'numpy.ndarray'>
Actual OUTPUT =  tensor([[2.3700],
        [4.6000],
        [9.4000],
        [2.2800]]) <class 'torch.Tensor'>
Predicted OUTPUT =  [[7.083707 ]
 [7.0839105]
 [7.084081 ]
 [7.0839186]] <class 'numpy.ndarray'>
Actual OUTPUT =  tensor([[0.7870],
        [1.9000],
        [5.8000],
        [1.4700]]) <class 'torch.Tensor'>
Predicted OUTPUT =  [[7.083986 ]
 [7.08321  ]
 [7.0840597]
 [7.0837936]] <class 'numpy.ndarray'>
Actual OUTPUT =  tensor([[ 9.4000],
        [ 0.5200],
        [ 1.8200],
        [30.6100]]) <class 'torch.Tensor'>
Predicted OUTPUT =  [[7.0838313]
 [7.0837   ]
 [7.083866 ]
 [7.0839286]] <class 'numpy.ndarray'>
Actual OUTPUT =  tensor([[1.2000],
        [1.0500],
        [0.8400],
        [0.8700]]) <class 'torch.Tensor'>
Predicted OUTPUT =  [[7.0838137]
 [7.084116 ]
 [7.0839806]
 [7.083787 ]] <class 'numpy.ndarray'>
Actual OUTPUT =  tensor([[ 5.7100],
        [ 7.1300],
    